In [17]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path

current_dir = Path.cwd()

# 取得上一層目錄 (專案根目錄，例如 D:\NCKU\paper4)
project_root = current_dir.parent

# 把根目錄加入 Python 的搜尋路徑
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    print(f"已加入專案路徑: {project_root}")

to_delete = [m for m in sys.modules if m.startswith('src')]

for m in to_delete:
    del sys.modules[m]

print(f"已清理模組: {to_delete}")

import src.utils.data_loader as data_loader
import src.utils.helpers as helpers
import src.engine.trainer as trainer
import src.engine.evaluator as evaluator

# 設定繪圖風格
sns.set_style("whitegrid")
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Microsoft JhengHei']  # 解決中文亂碼
plt.rcParams['axes.unicode_minus'] = False


已清理模組: ['src', 'src.utils', 'src.utils.data_loader', 'src.utils.helpers', 'src.engine', 'src.models', 'src.models.layers', 'src.models.network', 'src.utils.metrics', 'src.engine.trainer', 'src.engine.evaluator']


In [22]:
# 硬體設定
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {DEVICE}")

# 資料與模型參數
DATASET_PATH = Path("../dataset/USD_TWD.csv")
HORIZON = 3  # 預測未來 3 天
LOOKBACK = 30  # 回看過去 30 天
NUM_EPOCHS = 100  # 訓練輪數
LR = 0.001  # 學習率
SEED = 42  # 固定種子
CNNEXPERT_KERNELSIZE = 5
SERIESDECOMPOSITION_KERNELSIZE = 20

Using Device: cuda


In [19]:
if not DATASET_PATH.exists():
    print(f"[Error]: Dataset not found at {DATASET_PATH}")
else:
    print("[Loading Data]")
    df = pd.read_csv(DATASET_PATH)

    # 呼叫 src 裡的函式
    train_loader, test_loader, scalers_raw, _, _, _, _ = data_loader.prepare_data(
        df, lookback=LOOKBACK, horizon=HORIZON
    )

    # 檢查一下 Batch
    sample = next(iter(train_loader))
    print(f"[Loading Data] Data Loaded! Train Batches: {len(train_loader)}, Test Batches: {len(test_loader)}")
    print(f"[Loading Data] Input Shape: {sample['raw_input'].shape}, Target Shape: {sample['target'].shape}")

[Loading Data]
[Loading Data] Data Loaded! Train Batches: 167, Test Batches: 42
[Loading Data] Input Shape: torch.Size([32, 30, 2]), Target Shape: torch.Size([32, 3])


In [20]:
print(f"[Training] Starting Training for {NUM_EPOCHS} epochs...")

helpers.set_seed(42)

model = trainer.train_v11(
    train_loader=train_loader,
    test_loader=test_loader,
    device=DEVICE,
    horizon=HORIZON,
    num_epochs=NUM_EPOCHS,
    lr=LR,
    cnnExpert_KernelSize=CNNEXPERT_KERNELSIZE,
    seriesDecomposition_KernelSize=SERIESDECOMPOSITION_KERNELSIZE
)

print("[Training] Training Completed.")

[Training] Starting Training for 100 epochs...
Seed set to: 42 (Deterministic mode ON)

[Training] Enhanced DLinear...
  Epoch 20 | Loss: 0.5214 | Trend W: -0.029 | Seas W: 0.097
  Epoch 40 | Loss: 0.4745 | Trend W: -0.046 | Seas W: 0.121
  Epoch 60 | Loss: 0.4438 | Trend W: -0.058 | Seas W: 0.146
  Epoch 80 | Loss: 0.4163 | Trend W: -0.073 | Seas W: 0.163
  Epoch 100 | Loss: 0.4126 | Trend W: -0.075 | Seas W: 0.165
[Training] Training Completed.


In [21]:
print("[Evaluation] Running Evaluation...")

evaluator.evaluate_model(
    model=model,
    test_loader=test_loader,
    device=DEVICE,
    horizon=HORIZON
)

[Evaluation] Running Evaluation...

 FINAL MODEL EVALUATION (Horizon=3): Ablation Study (RevIN Enabled)
Metric               | Linear Base     | Base + CNN      | Improvement    
-----------------------------------------------------------------------------------------------
RMSE                 | 2.3108          | 1.8379          | -0.4728
R2 Score             | 0.5789          | 0.7336          | +0.1547
Avg Accuracy         | 0.5610          | 0.7080          | +0.1470
High Vol Accuracy    | 0.6933          | 0.8530          | +0.1597


{'RMSE_Final': np.float64(1.8379464054865469),
 'RMSE_Base': np.float64(2.3107838835608705),
 'R2_Final': 0.7335754632949829,
 'High_Vol_Acc': np.float64(0.85297845373891)}